In [ ]:
%pip install numpy
%pip install scipy
%pip install matplotlib
%pip install pyserial
%pip install sympy
%pip install sounddevice

In [ ]:
# For FFT using the pffft library, length must round as follows:
# N=(2^a)*(3^b)*(5^c), a >= 5, b >=0, c >= 0 (32, 48, 64, 96, 128, 144, 160, etc are all acceptable lengths)

for a in range(5, 15):
    for b in range(0, 10):
        for c in range(0, 10):
            num = 2**a*3**b*5**c
            if num > 7500 and num < 8500:
                print(f"a{a}b{b}c{c}: {2**a*3**b*5**c}")


In [ ]:
from sympy import factorint

Fs = 48000
Ts = 1 / Fs
# White noise length
noise_dur_s = 0.2
#Nw = int(noise_dur_s * Fs )
Nw = 5000
# Nw needs to be > 3000 (empirical tests)


# Rec length needs to be Nw + around 20m? worth of sample time
# (20m / 343 m/s) * 48000 = around 2800 samples
record_len = Nw + 2776

valid_correlation_len = record_len - Nw + 1
# length of noise will be padded with zeros to get the same size (to make the multiplication in FFT domain possible)

# For FFT using the pffft library, length must round as follows:
# N=(2^a)*(3^b)*(5^c), a >= 5, b >=0, c >= 0 (32, 48, 64, 96, 128, 144, 160, etc are all acceptable lengths)
factors = factorint(record_len)
print(factors)
assert(2 in factors)
assert(factors[2] >=5)
factors.pop(2, None)
factors.pop(3, None)
factors.pop(5, None)
assert(len(factors) == 0)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

# FIR filter
filter_order = 1000001 
cutoffs = [20000, 24000]
fir_coeff = sp.signal.firwin(filter_order, cutoffs[0], pass_zero='highpass', window='hann', fs=Fs)
#sp.signal.freqz(fir_coeff, plot=lambda w, h: plt.plot(w*Fs/np.pi/2, 10 * np.log10(np.abs(h))))
#plt.show()

## Generate noise
def make_noise(seed):
    rng = np.random.default_rng(seed)
    #white_noise = rng.normal(size=Nw)
    white_noise = rng.uniform(-1, 1, size=Nw)
    #white_noise = rng.integers(-1, 2, size=Nw)
    #white_noise = rng.integers(2, size=Nw)*2 - 1
    #white_noise = np.cbrt(rng.uniform(-1, 1, size=Nw))
    # plot filter
    

    # Applying the FIR filter to the white noise
    filtered_noise = sp.signal.convolve(white_noise, fir_coeff, mode='same')

    # plot noise
    #noise_fft = sp.fft.rfft(filtered_noise)
    #n=len(noise_fft)
    #f = np.arange(n)*Fs/n
    #plt.figure()
    #plt.plot(f, 10 * np.log10(np.abs(noise_fft)))
    #plt.show()

    return filtered_noise / max(filtered_noise)

def find_best():
    #best = [(349.86595152077996, 570), (341.6711598993898, 797), (339.4818560786098, 914), (333.3386605738086, 564)] normal cbrt
    #best = [(636.9564723016721, 564), (625.8389302895939, 661), (611.8865163886235, 711), (606.0413846722969, 932)] uniform
    best = [(0, -1)]*4
    for i in range(1000):
        noise = make_noise(i)
        m = np.sum(noise**2)
        for j in range(4):
            if m > best[j][0]:
                best[j] = (m, i)
                break
    print(best)
#find_best()
filtered_noise = make_noise(564)
filtered_noise2 = make_noise(661)
filtered_noise3 = make_noise(711)
filtered_noise4 = make_noise(932)

In [ ]:
np.save("filtered_noise1.npy", filtered_noise)
np.save("filtered_noise2.npy", filtered_noise2)
np.save("filtered_noise3.npy", filtered_noise3)
np.save("filtered_noise4.npy", filtered_noise4)

In [ ]:
for (i, noise) in enumerate([filtered_noise, filtered_noise2, filtered_noise3, filtered_noise4]):
    with open(f"pico_play_noise/sound_samples{i}.h", "w") as f:
        noise *= 2**15
        noise = np.asarray(noise, dtype='uint16')
        hexed = [f"0x{sample:04X}" for sample in noise]
        f.write("#include <stddef.h> // for size_t\n")
        f.write("#include <stdint.h> // for uint16_t\n")
        f.write("\n")
        f.write(f"#define SOUND_SAMPLES_LEN {len(hexed)}\n")
        f.write("\n")
        f.write("//These are actually signed ints \n")
        f.write("const uint16_t sound_samples[SOUND_SAMPLES_LEN] = {\n")
        for window in range(0, len(hexed), 10):
            s = "    " + ", ".join(hexed[window:window+10]) + ",\n"
            f.write(s)
        f.write("};\n")


In [ ]:
for (i, noise) in enumerate([filtered_noise, filtered_noise2, filtered_noise3, filtered_noise4]):
    with open(f"pico_record_noise/sound_correlation{i}.h", "w") as f:
        padded = np.pad(noise, (0, record_len-Nw), 'constant', constant_values=0)
        #print(len(padded))
        fft = np.fft.rfft(padded)
        #print(len(fft))
        conj = np.conj(fft)
        scaled = conj/np.max(np.abs(conj)) * 2**24
        # length of RFFT is N/2+1 complex numbers
        # We interleave the real and floats
        # Thus we have N+2 floats
        interleaved = np.zeros(len(padded)+2, dtype='uint32') 
        interleaved[0::2] = np.asarray(scaled.real, dtype='uint32')
        interleaved[1::2] = np.asarray(scaled.imag, dtype='uint32')
        interleaved = [f"0x{f:08X}" for f in interleaved]
        
        f.write("#include <stddef.h> // for size_t\n")
        f.write("\n")
        f.write(f"#define NUMBER_OF_SAMPLES_TO_RECORD {record_len}\n")
        f.write(f"#define NOISE_ACTUAL_LENGTH {Nw}\n")
        f.write(f"#define VALID_CORRELATION_LENGTH {valid_correlation_len}\n")
        f.write("\n")
        f.write(f"#define FFT_SIZE {record_len}\n")
        f.write(f"#define CONV_BUFFER_SIZE {record_len+2}\n")
        f.write("\n")
        f.write("uint32_t recorded_sound[CONV_BUFFER_SIZE] = {0};\n")
        f.write("const uint32_t sound_samples_fft_conj[CONV_BUFFER_SIZE] = {\n")
        for window in range(0, len(interleaved), 10):
            s = "    " + ", ".join(interleaved[window:window+10]) + ",\n"
            f.write(s)
        f.write("};\n")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

def correlate_and_find_delay(rec, noise):
    #rec_padded = np.pad(rec, (len(noise), 0), 'constant', constant_values=0)
    #print(rec)
    rec_fft = np.fft.rfft(rec)
    diff = len(rec)-len(noise)
    noise_padded = np.pad(noise, (0, diff), 'constant', constant_values=0)
    noise_fft_conj = np.conj(np.fft.rfft(noise_padded))
    #print(rec_fft.shape, noise_fft_conj.shape)
    cross_corr_freq = noise_fft_conj * rec_fft 
    cross_corr = np.abs(np.fft.irfft(cross_corr_freq))
    valid_len = diff + 1
    cross_corr = cross_corr[:valid_len]
    #print(cross_corr)
    plt.plot(cross_corr)
    plt.title("correlation")

    k_max_ind = np.argmax(cross_corr)

    return k_max_ind

Fs = 48000

recording_noise_1 = np.zeros(record_len)
recording_noise_2 = np.zeros(record_len)
recording_noise_3 = np.zeros(record_len)
recording_noise_4 = np.zeros(record_len)

insert_loc = 400
recording_noise_1[insert_loc:insert_loc+Nw] += filtered_noise
insert_loc2 = 1000
recording_noise_2[insert_loc2:insert_loc2+Nw] += filtered_noise2
insert_loc3 = 1200
recording_noise_3[insert_loc3:insert_loc3+Nw] += filtered_noise3
insert_loc4 = 1300
recording_noise_4[insert_loc4:insert_loc4+Nw] += filtered_noise4

recording_noise_together = recording_noise_1 + recording_noise_2 + recording_noise_3 + recording_noise_4
#recording_noise_together = recording_noise_together / max(recording_noise_together)

plt.figure()
ind1 = correlate_and_find_delay(recording_noise_1, filtered_noise)
plt.figure()
ind1 = correlate_and_find_delay(recording_noise_together, filtered_noise)
ind2 = correlate_and_find_delay(recording_noise_together, filtered_noise2)
ind3 = correlate_and_find_delay(recording_noise_together, filtered_noise3)
ind4 = correlate_and_find_delay(recording_noise_together, filtered_noise4)
print(ind1, ind2, ind3, ind4)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

thing = """"""

y = thing.splitlines()
print(y)
y = [abs(int(s)) for s in y]

plt.plot(y)


In [ ]:
import serial
import scipy as sp
import numpy as np 

fs, samples = sp.io.wavfile.read("song3.wav")
samples = samples // 32
print(fs, samples.dtype)
assert(samples.dtype == np.int16)
sample_bytes = samples.tobytes()

ser = serial.Serial("/dev/ttyACM0", 115200)
ser.write(sample_bytes)
ser.flush()
print("done")

In [ ]:
import serial
import scipy as sp
import numpy as np 

ser = serial.Serial("/dev/ttyACM1", 115200)
ser.write(b"r")
b = ser.read(48000 * 2 * 5)

#samples = np.frombuffer(b, dtype=np.uint32)
#samples = (samples)
#samples = np.frombuffer(samples.tobytes(), dtype=np.int32)
samples = np.frombuffer(b, dtype=np.int16)
import sounddevice as sd

sd.play(samples, 48000)
sd.wait()
for i in samples:
    print(hex(i))